# Pandas
## Data 765 tutoring

[pandas](https://pandas.pydata.org/) is an in memory tabular data frame library written in Python, Cython, and C and builds upon [NumPy](https://numpy.org/). `pandas` loads and manipulates data _in memory_ rather than storing temporary files or loading and operating on chunks. `pandas`' objects are tables that are closer to relational databases (i.e. SQL) than free form and nested objects (i.e. nested JSON or NoSQL). Data are stored as columns rather than rows. Columns are more efficient than rows due to data locality (i.e. what we spoke about in the NumPy mini lecture).

Data frames are omnipresent data structures in data analysis. Data frames are more than just abstractions over efficient math. They implement features to ease reshaping data or calculating metrics. For example, exploding a `pandas` DataFrame, joining DataFrames, or aggregating are all made easier by `pandas`.

`pandas` is a general purpose data frame library that fits a wide range of use cases. Other libraries implement specialized data frames that may be optimized for different workloads.

* [Polars](https://www.pola.rs/) is a speedy data frame library written in Rust with Python bindings. Polars is built on [Apache Arrow](https://github.com/apache/arrow-rs), a development framework for in memory data.
* [Apache Spark](https://spark.apache.org/) is designed for distributed, scalable workloads. Spark isn't written in Python, but it has bindings for Python, Scala, Java, R, and others.
* [Dask](https://dask.org/)'s API is similar to `pandas`. `Dask` scales up to clusters but scales down to single machines as well. `Dask` is useful for analyzing big data that don't fit into memory. 
* [datatable](https://github.com/h2oai/datatable) is a data frame library that is similar to R's [data.table](https://rdatatable.gitlab.io/data.table/). `datatable` uses optimized algorithms written in C that are based on `data.table`'s routines.

The above list isn't exhaustive but you get the idea.

# Data ingestion

`pandas` API is pretty intuitive and easy to use. The API itself is extensive which means that exhaustively covering everything in a few weeks is impossible. Instead, like the class lectures, I'll explain how to navigate and use the API so that you're able to waltz around the docs without being utterly overwhelmed.

Speaking of which: [the docs](https://pandas.pydata.org/docs/)

Don't read the docs end to end. Documentation should be purused for specific topics. Libraries usually have quick start guides or longer usage guides that can help with idiomatic use.

I'll use Pokémon data again...of course. I wrote a small scraper for one endpoint of the PokeAPI to gather data for these next few weeks. I didn't finish it yet though, so I'm using [data from Kaggle](https://www.kaggle.com/mariotormo/complete-pokemon-dataset-updated-090420) as a temp.

In [1]:
import pandas as pd

pokemon = pd.read_csv("pokedex.csv")

`pandas` is typically imported as the alias `pd`. You can load several dreadful formats such as Excel, SAS, or SPSS as well as sane formats like CSV, [Parquet](https://en.wikipedia.org/wiki/Apache_Parquet), or [HDF](https://en.wikipedia.org/wiki/Hierarchical_Data_Format).

Generally speaking, you can look up "pandas FILEFORMAT" in your search engine of choice to find the function you need quickly. The documentation also has a [page listing the functions](https://pandas.pydata.org/pandas-docs/stable/user_guide/io.html). `pandas` can write to supported formats as well. A general paradigm is to clean up and reshape data then write the table out to a format for reuse.

`pandas`' I/O is flexible. Local files, remote files, file pointers, et cetera can be used to ingest data. For example, you can load the data I scraped for my thesis right from GitHub without saving a local copy.

In [ ]:
gamers = pd.read_csv("https://github.com/joshuamegnauth54/GamerDistributionThesis2020/raw/master/data/gamers_reddit_medium_2020.csv")

Or you can execute a SQL query on a server and create a DataFrame from the result.

Don't run this cell because the connection will fail.

In [ ]:
import psycopg2

with psycopg2.connect(database="pokemon", 
                      user="joshua",
                      password="fakepasswordilikecats",
                      host="localhost",
                      port="20000") as conn:
    
    query = """
            WITH pokemon_high_offense(species,
                          id,
                          base_stats,
                          ptype,
                          sp_atk,
                          sp_def,
                          atk,
                          def,
                          speed,
                          hp)
            AS (
                SELECT species,
                       id,
                       base_stats,
                       sp_atk,
                       sp_def,
                       atk,
                       def,
                       speed,
                       hp
                FROM pokemon
                WHERE (sp_atk >= percentile_cont(.5) WITHIN GROUP (ORDER BY sp_atk))
                       OR 
                       (atk >= percentile_cont(.5) WITHIN GROUP (ORDER BY atk))
                )
            SELECT *
            FROM pokemon_high_offense;
            """
    poke_hi_off = pd.read_sql(query, conn)

Each of these I/O functions have many parameters to tweak the ingestion process. For example, [pd.read_csv()](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.read_csv.html) supports:

* `delimiter`: providing a delimiter in case the engine fails to detect one
* `header`: specifying a different row to use as the header (column names)
* `names`: set column names
* `index_col`: column to use an index
* `usecols`: keep specific columns
* Parameters for parsing dates.

And **much** more. You should definitely take a look at the function docs for loading data as you will invariably have to specify parameters in the future.

# Series and DataFrames
[Series](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.html) and [DataFrame](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) are `pandas`' main data structures. `DataFrames` consist of `Series`. `Series` are one dimensional ndarrays with additional exposed features to ease data ops.

`DataFrame`, as the documentation mentions, is a container for one or more `Series`. `DataFrame`s are more than a collection of ndarrays or `Series`. Recall the simple table example from the [data structures](https://github.com/joshuamegnauth54/data765-intro-python-tutoring/blob/main/notebooks/02-collections_basics.ipynb) mini-lecture. `DataFrame`s ease working with a similiar structure. For example, how would you calculate row-wise or column-wise statistics? You'd have to traverse a nested `list` by row or row and column in other to calculate a mean. This gets messy fast. Now imagine group bys, handling nulls, converting types, et cetera.